In [1]:
import etl_utils as eu
import pandas as pd 
import os

In [2]:
STAGING_1 = r"..\data\alternative_data\staging_1"
STAGING_2 = r"..\\data\\alternative_data\\staging_2"

In [3]:
csv_files = eu.get_csv_files(STAGING_1)

In [ ]:
google_trends = "cali-monthly-google_trends.csv"
mobility_report = "us-daily-mobility_report.csv"
consumer_sentiment = "us-monthly-consumer_sentiment.csv"

for file in csv_files:
    file_name = os.path.basename(file)
    data = pd.read_csv(file)
    
    if file_name == google_trends:
        
        data = data.replace("<1", "0")  # replacing unformatted numbers
        
    elif file_name == mobility_report:
        
        date_col = data.pop('date')  
        data.insert(0, 'date', date_col)   # querying file for california data 
        data = data.query("state == 'California'")
        
        file = eu.get_modified_file_path(file_name, "cali-daily-mobility_report.csv")
    
    elif file_name == consumer_sentiment:
        
        data[["Month", "Year"]] = data[["Month", "Year"]].astype(str) # adjusting date string 
        date_series = data['Year'] + "-" + data['Month']
        data.insert(0, "date", date_series)

    file = eu.change_directory(file, STAGING_2)
    data.to_csv(file, index=False)   

    

In [4]:
data = pd.read_csv(r"D:\articles\alternative_economic_data\data\alternative_data\staging_2\cali-daily-mobility_report.csv")

In [6]:
eu.transform_to_date(data)

ParserError: Unknown string format: California

In [7]:
eu.transform_to_quarterly(data, "cali-daily-mobility_report.csv")

,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential
date,,,,,,
2020-01-01,-9.663043,1.121679,5.635752,-13.820553,-13.814385,5.749154
2020-04-01,-34.752875,-4.274603,1.076112,-30.428645,-34.997269,13.604072
2020-07-01,-21.068781,-3.042320,27.374552,-23.665104,-30.246702,9.184574
2020-10-01,-21.566742,-6.362448,-5.857212,-31.145487,-29.084831,9.878284
2021-01-01,-21.962792,-10.148420,-9.084249,-33.968847,-28.401970,10.155331
2021-04-01,-6.061940,1.098252,23.871578,-20.542243,-25.104903,5.559176
2021-07-01,-3.890128,2.993208,23.681255,-22.607630,-25.248889,5.046767
2021-10-01,-8.080903,-0.308904,-5.576222,-26.968262,-24.786307,6.004712
2022-01-01,-16.963743,-7.199623,-8.820600,-33.718341,-24.905765,8.410390
